In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
# Hyperparameter

learning_rate = 1e-3
batch_size = 64
epochs = 5

### Optimization

- train : train data set을 반복하기 최적의 매개변수로 수렴
- val/test : 모델 성능이 개선되는지 확인

In [3]:
# Loss function

loss_fn = nn.CrossEntropyLoss()

In [4]:
# Optimizer

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [5]:
# 최적화 코드 반복 수행

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306418  [    0/60000]
loss: 2.288605  [ 6400/60000]
loss: 2.268543  [12800/60000]
loss: 2.264717  [19200/60000]
loss: 2.248932  [25600/60000]
loss: 2.220107  [32000/60000]
loss: 2.239670  [38400/60000]
loss: 2.202075  [44800/60000]
loss: 2.193107  [51200/60000]
loss: 2.167961  [57600/60000]
Test Error: 
 Accuracy: 39.1%, Avg loss: 2.157888 

Epoch 2
-------------------------------
loss: 2.169214  [    0/60000]
loss: 2.153880  [ 6400/60000]
loss: 2.096186  [12800/60000]
loss: 2.115786  [19200/60000]
loss: 2.071803  [25600/60000]
loss: 2.010020  [32000/60000]
loss: 2.052829  [38400/60000]
loss: 1.967209  [44800/60000]
loss: 1.972768  [51200/60000]
loss: 1.907811  [57600/60000]
Test Error: 
 Accuracy: 55.9%, Avg loss: 1.899934 

Epoch 3
-------------------------------
loss: 1.930485  [    0/60000]
loss: 1.894311  [ 6400/60000]
loss: 1.778434  [12800/60000]
loss: 1.826398  [19200/60000]
loss: 1.723243  [25600/60000]
loss: 1.668512  [32000/600

Epoch마다 Accuracy는 높아지고 loss는 낮아짐